## Crime Rate Prediction based on Lunar Cycle

#### Purpose:
To predict the crime rate for days closer to the Full Moon Days for any given year.

#### Assumption:
1. The thesis that proves the existance of relationship between Lunar Cycle and Criminal behavior of human beings is true.
2. The Lunar Day information obtained from the website https://www.timeanddate.com is accurate.
3. The Census information obtained from the website https://www.google.com/publicdata/explore?ds=kf7tgg1uo9ude is accurate.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import warnings;
warnings.simplefilter('ignore')

In [3]:
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Do The following steps for scraping Lunar Cycle Data from www.timeanddate.com <br/><br/>
Step#1: Open Anaconda Command Prompt<br/><br/>
Step#2: Run the below command<br/>
<p>
*        scrapy runspider C:\Users\dhars\Documents\GL_ML\Capstone\Crime_Rate_Prediction_And_Lunar_Cycle\Code\Web_Scrapping\Lunar_Cycle_WebScrapper_v3.py -o output.json</p><br/>

Step#3: When the code prompts you for user input, provide the name of the city and country along with the years between which you need the lunar data.<br/>
<p>
*        Eg. City: Atlanta<br/>
*        Country: USA<br/>
*        Starting Year: 2009<br/>
*        Ending Year: 2018
</p><br/>

Step#4: The file Lunar_Cycle_2009_2018.csv will be created in the below location<br/>
*     "C:\Users\dhars\Documents\GL_ML\Capstone\Crime_Rate_Prediction_And_Lunar_Cycle\Code\Web_Scrapping"<br/>

In [5]:
Lunar_Cycle = pd.read_csv('Lunar_Cycle_2009_2018.csv')

In [6]:
np.sort(Lunar_Cycle.Full_Moon.unique())

array(['01-Aug', '01-Jan', '01-Jul', '01-Mar', '02-Dec', '02-Jun',
       '02-Nov', '03-Dec', '03-Feb', '03-Jul', '03-May', '04-Apr',
       '04-Jan', '04-Jun', '04-Nov', '04-Oct', '04-Sep', '05-Aug',
       '05-Mar', '05-May', '05-Oct', '06-Apr', '06-Dec', '06-Nov',
       '06-Sep', '07-Aug', '07-Feb', '07-Jul', '07-Jun', '08-Mar',
       '08-Oct', '08-Sep', '09-Apr', '09-Feb', '09-Jan', '09-Jul',
       '09-Jun', '09-May', '10-Aug', '10-Dec', '10-Feb', '10-Jan',
       '10-Mar', '10-May', '10-Nov', '11-Apr', '11-Oct', '12-Dec',
       '12-Jan', '12-Jul', '12-Mar', '12-Sep', '13-Aug', '13-Dec',
       '13-Jun', '13-Nov', '14-Feb', '14-May', '14-Nov', '14-Oct',
       '15-Apr', '15-Jan', '15-Jul', '15-Jun', '15-Sep', '16-Aug',
       '16-Mar', '16-Oct', '16-Sep', '17-Apr', '17-Dec', '17-May',
       '17-Nov', '18-Aug', '18-Feb', '18-Jul', '18-Jun', '18-Oct',
       '19-Jan', '19-Jul', '19-Mar', '19-May', '19-Sep', '20-Apr',
       '20-Aug', '20-Feb', '20-Jun', '21-Dec', '21-Mar', '21-M

In [7]:
Lunar_Cycle.drop(index=Lunar_Cycle[Lunar_Cycle.Full_Moon=='\xa0'].index,inplace=True)
Lunar_Cycle.Full_Moon=Lunar_Cycle.Full_Moon.str.cat(Lunar_Cycle.Year.astype('str'),'-')
Lunar_Cycle.Full_Moon=pd.to_datetime(Lunar_Cycle.Full_Moon)
Lunar_Cycle.drop(columns=['SL','Lunation'],inplace=True)
Lunar_Cycle.head()

,Year,Full_Moon
0,2008,2008-01-22
1,2008,2008-02-20
2,2008,2008-03-21
3,2008,2008-04-20
4,2008,2008-05-19


In [8]:
Calender = pd.DataFrame(columns=['Year','Date'])
Calender.Date = pd.date_range(start='1/1/2009', end='31/12/2018')
Calender.Year = Calender.Date.dt.year
Calender['Lunar_Day']=0
Calender = Calender.merge(Lunar_Cycle,how='left',left_on=Calender['Date'],right_on=Lunar_Cycle.Full_Moon)
Calender['Year'] = Calender['Year_x']
Calender.drop(columns=['key_0','Year_x','Year_y'],inplace=True)
Calender.Full_Moon = pd.to_datetime(Calender.Full_Moon)
Calender.loc[Calender[Calender.Full_Moon.notna()].index,'Lunar_Day'] = 1
Calender.head()

,Date,Lunar_Day,Full_Moon,Year
0,2009-01-01,0,NaT,2009
1,2009-01-02,0,NaT,2009
2,2009-01-03,0,NaT,2009
3,2009-01-04,0,NaT,2009
4,2009-01-05,0,NaT,2009


In [9]:
for i in range(Calender.Year.count()-1):
    if ((Calender.loc[i,'Lunar_Day'] != 0)&(Calender.loc[i+1,'Lunar_Day'] != 1)):
        Calender.loc[i+1,'Lunar_Day'] = Calender.loc[i,'Lunar_Day'] + 1

In [10]:
prev_lunar = Lunar_Cycle[Lunar_Cycle.Year == Calender[Calender.Lunar_Day == 0].Year.unique()[0]-1].Full_Moon.dt.date.values[-1]
Calender.loc[Calender[Calender.Lunar_Day == 0].index,'Lunar_Day'] = (Calender[Calender.Lunar_Day == 0].Date.dt.date - prev_lunar).dt.days
Calender.drop(columns=['Full_Moon','Year'],inplace=True)
Calender.head(15)

,Date,Lunar_Day
0,2009-01-01,20
1,2009-01-02,21
2,2009-01-03,22
3,2009-01-04,23
4,2009-01-05,24
5,2009-01-06,25
6,2009-01-07,26
7,2009-01-08,27
8,2009-01-09,28
9,2009-01-10,1


In [11]:
CrimeReport = pd.read_csv('Atlanta_2009_updated.csv')

In [12]:
CrimeReport.head().transpose()

,0,1,2,3,4
Report_Number,90010061,90020500,153221163,90010939,90020008
Report_Date,01-01-2009,02-01-2009,18-11-2015,01-01-2009,02-01-2009
Occur_Date,01-01-2009,01-01-2009,01-01-2009,01-01-2009,01-01-2009
Occur_Time,'0010,'1800,'0000,'0200,'1630
Possible_Date,01-01-2009,02-01-2009,19-09-2015,01-01-2009,02-01-2009
Possible_Time,'0020,'0600,'1525,'1154,'0002
Beat,309,309,414,408,309
Apartment_Office_Prefix,NaN,NaN,NaN,NaN,NaN
Apartment_Number,NaN,NaN,NaN,R-2,517
Location,501 TUFTON TRL SE,3065 BROWNS MILL RD SE,3301 N CAMP CREEK,3000 STONE HOGAN CONN SW,2762 VINEYARDS DR SE


In [13]:
CrimeReport.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Report_Number,39327.0,9.192189e+07,1.361731e+06,6.168198e+07,9.099223e+07,9.187128e+07,9.277086e+07,1.532212e+08
Beat,39307.0,3.696070e+02,1.655563e+02,1.010000e+02,2.080000e+02,4.060000e+02,5.060000e+02,6.120000e+02
UCR_#,39327.0,5.886044e+02,1.128458e+02,1.100000e+02,5.110000e+02,6.400000e+02,6.700000e+02,7.300000e+02
Latitude,39327.0,3.375626e+01,4.387195e-02,3.364835e+01,3.372975e+01,3.375605e+01,3.377953e+01,3.388606e+01
Longitude,39327.0,-8.440640e+01,4.773341e-02,-8.455041e+01,-8.443276e+01,-8.439517e+01,-8.437206e+01,-8.429013e+01


In [14]:
CrimeReport['Lat1']=np.round(CrimeReport.Latitude,1)
CrimeReport['Long1']=np.round(CrimeReport.Longitude,1)
CrimeReport['Coordinates1']=list(zip(CrimeReport.Lat1,CrimeReport.Long1))
CrimeReport['Lat3']=np.round(CrimeReport.Latitude,3)
CrimeReport['Long3']=np.round(CrimeReport.Longitude,3)
CrimeReport['Coordinates3']=list(zip(CrimeReport.Lat3,CrimeReport.Long3))
CrimeReport['Occur_Date']=pd.to_datetime(CrimeReport['Occur_Date'])

In [15]:
CrimeReport=CrimeReport.merge(Calender,how='left',left_on=CrimeReport.Occur_Date,right_on=Calender.Date)
CrimeReport.drop(columns=['key_0','Date'],inplace=True)
CrimeReport.head()

,Report_Number,Report_Date,Occur_Date,Occur_Time,Possible_Date,Possible_Time,Beat,Apartment_Office_Prefix,Apartment_Number,Location,...,NPU,Latitude,Longitude,Lat1,Long1,Coordinates1,Lat3,Long3,Coordinates3,Lunar_Day
0,90010061,01-01-2009,2009-01-01,'0010,01-01-2009,'0020,309.0,NaN,NaN,501 TUFTON TRL SE,...,Z,33.66477,-84.38430,33.7,-84.4,"(33.7, -84.4)",33.665,-84.384,"(33.665, -84.384)",20
1,90020500,02-01-2009,2009-01-01,'1800,02-01-2009,'0600,309.0,NaN,NaN,3065 BROWNS MILL RD SE,...,Z,33.67107,-84.38240,33.7,-84.4,"(33.7, -84.4)",33.671,-84.382,"(33.671, -84.382)",20
2,153221163,18-11-2015,2009-01-01,'0000,19-09-2015,'1525,414.0,NaN,NaN,3301 N CAMP CREEK,...,P,33.67210,-84.50122,33.7,-84.5,"(33.7, -84.5)",33.672,-84.501,"(33.672, -84.501)",20
3,90010939,01-01-2009,2009-01-01,'0200,01-01-2009,'1154,408.0,NaN,R-2,3000 STONE HOGAN CONN SW,...,R,33.67424,-84.49697,33.7,-84.5,"(33.7, -84.5)",33.674,-84.497,"(33.674, -84.497)",20
4,90020008,02-01-2009,2009-01-01,'1630,02-01-2009,'0002,309.0,NaN,517,2762 VINEYARDS DR SE,...,Z,33.67957,-84.36996,33.7,-84.4,"(33.7, -84.4)",33.680,-84.370,"(33.68, -84.37)",20


In [16]:
CrimeReport.head().transpose()

,0,1,2,3,4
Report_Number,90010061,90020500,153221163,90010939,90020008
Report_Date,01-01-2009,02-01-2009,18-11-2015,01-01-2009,02-01-2009
Occur_Date,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00,2009-01-01 00:00:00
Occur_Time,'0010,'1800,'0000,'0200,'1630
Possible_Date,01-01-2009,02-01-2009,19-09-2015,01-01-2009,02-01-2009
Possible_Time,'0020,'0600,'1525,'1154,'0002
Beat,309,309,414,408,309
Apartment_Office_Prefix,NaN,NaN,NaN,NaN,NaN
Apartment_Number,NaN,NaN,NaN,R-2,517
Location,501 TUFTON TRL SE,3065 BROWNS MILL RD SE,3301 N CAMP CREEK,3000 STONE HOGAN CONN SW,2762 VINEYARDS DR SE


In [17]:
UCR=CrimeReport.loc[:,['UCR_Literal','UCR_#']]
UCR=UCR.drop_duplicates().sort_values(by=['UCR_Literal','UCR_#']).reset_index().drop(columns='index')
UCR.head()

,UCR_Literal,UCR_#
0,AGG ASSAULT,410
1,AGG ASSAULT,420
2,AGG ASSAULT,430
3,AGG ASSAULT,440
4,AUTO THEFT,710


In [18]:
CrimeReport1=CrimeReport.drop(columns=['Apartment_Number',
'Apartment_Office_Prefix',
'Beat',
'Latitude','Lat1','Lat3',
'Location',
'Longitude','Long1','Long3',
'Occur_Time',
'Possible_Date',
'Possible_Time',
'Report_Date',
'Report_Number',
'Shift_Occurence',
'UCR_Literal'])

In [19]:
CrimeReport1.head()

,Occur_Date,Location_Type,UCR_#,IBR_Code,Neighborhood,NPU,Coordinates1,Coordinates3,Lunar_Day
0,2009-01-01,20,511,2202,Glenrose Heights,Z,"(33.7, -84.4)","(33.665, -84.384)",20
1,2009-01-01,NaN,511,2202,Glenrose Heights,Z,"(33.7, -84.4)","(33.671, -84.382)",20
2,2009-01-01,26,410,1314,NaN,P,"(33.7, -84.5)","(33.672, -84.501)",20
3,2009-01-01,18,710,2404,Greenbriar,R,"(33.7, -84.5)","(33.674, -84.497)",20
4,2009-01-01,18,720,2424,Rosedale Heights,Z,"(33.7, -84.4)","(33.68, -84.37)",20


In [20]:
CrimeReport1.dtypes

Occur_Date       datetime64[ns]
Location_Type            object
UCR_#                     int64
IBR_Code                 object
Neighborhood             object
NPU                      object
Coordinates1             object
Coordinates3             object
Lunar_Day                 int64
dtype: object

In [21]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()


Coord1=pd.DataFrame()
Coord1['Coordinates1']=CrimeReport1['Coordinates1']
Coord1['Categories']=lb_make.fit_transform(CrimeReport1['Coordinates1'].astype(str))
CrimeReport1['Coordinates1']=lb_make.fit_transform(CrimeReport1['Coordinates1'].astype(str))
Coord1=Coord1.drop_duplicates().sort_values(by='Categories').reset_index(drop=True)
Coord1.head()

Coord3=pd.DataFrame()
Coord3['Coordinates3']=CrimeReport1['Coordinates3']
Coord3['Categories']=lb_make.fit_transform(CrimeReport1['Coordinates3'].astype(str))
CrimeReport1['Coordinates3']=lb_make.fit_transform(CrimeReport1['Coordinates3'].astype(str))
Coord3=Coord3.drop_duplicates().sort_values(by='Categories').reset_index(drop=True)
Coord3.head()

,Coordinates1,Categories
0,"(33.6, -84.4)",0
1,"(33.7, -84.3)",1
2,"(33.7, -84.4)",2
3,"(33.7, -84.5)",3
4,"(33.7, -84.6)",4


,Coordinates3,Categories
0,"(33.648, -84.364)",0
1,"(33.648, -84.366)",1
2,"(33.649, -84.356)",2
3,"(33.649, -84.358)",3
4,"(33.649, -84.36)",4


In [22]:
CrimeReport1.head()

,Occur_Date,Location_Type,UCR_#,IBR_Code,Neighborhood,NPU,Coordinates1,Coordinates3,Lunar_Day
0,2009-01-01,20,511,2202,Glenrose Heights,Z,2,174,20
1,2009-01-01,NaN,511,2202,Glenrose Heights,Z,2,287,20
2,2009-01-01,26,410,1314,NaN,P,3,312,20
3,2009-01-01,18,710,2404,Greenbriar,R,3,372,20
4,2009-01-01,18,720,2424,Rosedale Heights,Z,2,512,20


In [23]:
CrimeReport1.dtypes

Occur_Date       datetime64[ns]
Location_Type            object
UCR_#                     int64
IBR_Code                 object
Neighborhood             object
NPU                      object
Coordinates1              int32
Coordinates3              int32
Lunar_Day                 int64
dtype: object

In [24]:
CrimeReport2=pd.DataFrame(data=CrimeReport1.groupby(by=['Occur_Date','Coordinates1','Coordinates3','Lunar_Day']).count()['UCR_#'].values,
                   index=CrimeReport1.groupby(by=['Occur_Date','Coordinates1','Coordinates3','Lunar_Day']).count()['UCR_#'].index)
CrimeReport2=CrimeReport2.reset_index()
CrimeReport2=CrimeReport2.reset_index(drop=True)
CrimeReport2['Crime_Count'] = CrimeReport2.loc[:,0]
CrimeReport2=CrimeReport2.drop(columns=0)
CrimeReport2.head()

,Occur_Date,Coordinates1,Coordinates3,Lunar_Day,Crime_Count
0,2009-01-01,1,3170,20,1
1,2009-01-01,1,3421,20,1
2,2009-01-01,1,4230,20,1
3,2009-01-01,2,174,20,1
4,2009-01-01,2,287,20,1


In [25]:
CrimeReport2['Occur_Date']=lb_make.fit_transform(CrimeReport2['Occur_Date'])
CrimeReport2=CrimeReport2[((CrimeReport2.Lunar_Day.between(25,30)) | (CrimeReport2.Lunar_Day.between(1,5)))]

In [26]:
CrimeReport2['Crime_Rate']=(CrimeReport2.Crime_Count*100000)/483450
CrimeReport2.sort_values(by='Crime_Rate',ascending=False).head()

,Occur_Date,Coordinates1,Coordinates3,Lunar_Day,Crime_Count,Crime_Rate
37549,363,6,10317,29,9,1.861620
34360,330,6,10317,26,7,1.447926
37351,361,6,10317,27,6,1.241080
37192,359,6,10317,25,6,1.241080
22958,219,6,10317,4,5,1.034233


In [27]:
#CrimeReport2.plot.scatter(x='Lunar_Day',y='Crime_Count',c='Coordinates1')

In [28]:
#sns.pairplot(data=CrimeReport2,hue='Crime_Count',size=5)

In [29]:
CrimeReport2.sort_values(by='Crime_Count',ascending=False).head()

,Occur_Date,Coordinates1,Coordinates3,Lunar_Day,Crime_Count,Crime_Rate
37549,363,6,10317,29,9,1.861620
34360,330,6,10317,26,7,1.447926
37351,361,6,10317,27,6,1.241080
37192,359,6,10317,25,6,1.241080
22958,219,6,10317,4,5,1.034233


In [30]:
x=CrimeReport2.drop(columns=['Crime_Count','Crime_Rate'])
y=CrimeReport2.Crime_Count

In [31]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [32]:
x_train.reset_index(inplace=True,drop=True)
y_train.reset_index(inplace=True,drop=True)
x_test.reset_index(inplace=True,drop=True)
y_test.reset_index(inplace=True,drop=True)

In [33]:
from sklearn.svm import SVC
import multiprocessing 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,recall_score
from sklearn.model_selection import cross_val_score

In [34]:
param_grid = [    {        
     'kernel': ['rbf'],        
     'C': [0.01,0.1,0.5,1]
              } ]

In [35]:
x_train.dtypes

Occur_Date      int64
Coordinates1    int64
Coordinates3    int64
Lunar_Day       int64
dtype: object

In [36]:
gs = GridSearchCV(estimator=SVC(), param_grid=param_grid,scoring='accuracy', cv=10, n_jobs=multiprocessing.cpu_count())

In [37]:
gs.fit(x_train, y_train) 

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=8,
       param_grid=[{'kernel': ['rbf'], 'C': [0.01, 0.1, 0.5, 1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [38]:
gs.best_estimator_

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [39]:
gs.best_score_ 

0.9632375979112272

In [56]:
SC = SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)




In [58]:
SC.fit(x_train,y_train)


SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [63]:
y_predict = SC.predict(x_test)
score = SC.score(x_test,y_test)
#train_score = SC.score(y_train,y_pred)
print('Score',score)
#print('Score',train_score)
print('Confusion Matrix \n',confusion_matrix(y_test, y_predict))
print('Recall Score \n',recall_score(y_test, y_predict, average = 'micro'))

Score 0.9634502923976608
Confusion Matrix 
 [[3954    0    0    0    0    0]
 [ 120    0    0    0    0    0]
 [  23    1    0    0    0    0]
 [   4    0    0    0    0    0]
 [   1    0    0    0    0    0]
 [   1    0    0    0    0    0]]
Recall Score 
 0.9634502923976608


y_pred=RFReg_Model.predict(x_test)
y_pred=np.round(y_pred,0)
print('Explained Variance Score:',round(met.regression.explained_variance_score(y_test,y_pred),3))
print('Mean Absolute Error:',round(met.regression.mean_absolute_error(y_test,y_pred),3))
print('Mean Squared Error:',round(met.regression.mean_squared_error(y_test,y_pred),3))
print('Root Mean Squared Error:',round(np.sqrt(met.regression.mean_squared_error(y_test,y_pred)),3))
print('Mean Squared Log Error:',round(met.regression.mean_squared_log_error(y_test,y_pred),3))
print('Median Absolute Error:',round(met.regression.median_absolute_error(y_test,y_pred),3))
print('R2 Score:',round(met.regression.r2_score(y_test,y_pred),3))
print('Test Accuracy Score:',round(met.accuracy_score(y_test,y_pred),3))
print('Confusion Matrix:\n',met.confusion_matrix(y_test,y_pred))
precision, recall, f_score, support = met.classification.precision_recall_fscore_support(y_test,y_pred)
prf = pd.DataFrame()
prf['Crime_Count']=y_test.unique().flatten()
prf['Precision']=precision.flatten()
prf['Recall']=recall.flatten()
prf['Fscore']=f_score.flatten()
prf['Support']=support.flatten()
print('\nPrecision Recall FScore Support Matrix:\n')
prf
y_pred=DTR_Model.predict(x_train)
y_pred=np.round(y_pred,0)
print('Train Accuracy Score:',round(met.accuracy_score(y_train,y_pred),3))
print('Confusion Matrix:\n',met.confusion_matrix(y_train,y_pred))
precision, recall, f_score, support = met.classification.precision_recall_fscore_support(y_train,y_pred)
prf = pd.DataFrame()
prf['Crime_Count']=y_train.unique().flatten()
prf['Precision']=precision.flatten()
prf['Recall']=recall.flatten()
prf['Fscore']=f_score.flatten()
prf['Support']=support.flatten()
print('\nPrecision Recall FScore Support Matrix:\n')
prf